In [33]:
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, roc_curve, auc


In [34]:
churners_df = pd.read_csv("../../data/BankChurners.csv", index_col='CLIENTNUM')

print(churners_df.shape)

# # Assuming 'df' is your DataFrame containing the dataset
# current_month = 2024 * 12 + 5  # Convert reference date to months
# df['Customer_Tenure'] = current_month - df['Months_on_book']

(10127, 20)


In [35]:
def adding_features(df):
    return df

In [36]:
def imputer(X_train, X_test):
    # Create an empty list to store columns with 'unknown' values
    columns_train_with_unknown = []
    columns_test_with_unknown = []

    # Iterate over each column
    for col in X_train.columns:
        # Check if any value in the column is 'unknown'
        if 'Unknown' in X_train[col].values:
            # If 'unknown' value is found, add the column name to the list
            columns_train_with_unknown.append(col)
    
    # Iterate over each column
    for col in X_test.columns:
        # Check if any value in the column is 'unknown'
        if  'Unknown' in X_test[col].values:
            # If 'unknown' value is found, add the column name to the list
            columns_test_with_unknown.append(col)   

In [37]:
def encode(X_train, X_test):
    # encode categorical features
    categorical_cols = X_train.select_dtypes('object').columns

    encoder = LabelEncoder()
    for col in categorical_cols:
        X_train[col] = encoder.fit_transform(X_train[col])
        
        #If a value is encountered in the test data that wasn't seen during training (i.e., not present in the label encoder's classes), 
        #it's assigned a placeholder value of -1. This ensures that unseen categorical values in the test data are handled appropriately 
        #during the encoding process.
        X_test[col] = X_test[col].map(lambda s: encoder.transform([s])[0] if s in encoder.classes_ else -1)
    
    return X_train, X_test

In [38]:
def scaler(X_train, X_test):
    return X_train, X_test

In [39]:
def preprocess(df):

    # no missing values --> imputing = not necessary
    df.isnull().sum()
    adding_features(df)

    X = churners_df.drop('Attrition_Flag', axis=1)
    y = churners_df['Attrition_Flag']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    X_train, X_test = imputer(X_train, X_test)
    X_train, X_test = encode(X_train, X_test)
    X_train, X_test = scaler(X_train, X_test)

    return X_train, X_test, y_train, y_test


In [40]:
preprocess(churners_df)

TypeError: cannot unpack non-iterable NoneType object